In [1]:
%load_ext autoreload
%autoreload 2

## Parameters

In [2]:
# Set parameters
dimensionality_reduction = 'NMF'
dimension = 2
layers = ['arr_0', 'arr_1', 'arr_2', 'arr_3', 'arr_4', 'arr_5', 'arr_6', 'arr_7', 'arr_8', 'arr_9', 'arr_10', 'arr_11', 'arr_12']  # which layers to visualize
# layers = ['arr_0']  # good for debugging


In [3]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle
import numpy as np
import os
from lucid.misc.io.showing import _display_html
import sys
project_path = os.path.abspath('../..')
sys.path.insert(0, project_path)
from src.utils import acts_util, vis_util

# import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased', output_hidden_states=True)


In [ ]:
sents = ["Hello, my dog is cute.", 'I am happy!']

inputs = tokenizer(*sents, return_tensors="pt")
outputs = model(**inputs)

hidden_states = outputs.hidden_states
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)

tokens = [tokenizer.decode(i).replace(' ', '') for i in inputs['input_ids'].tolist()[0]]
acts = token_embeddings

layer_to_acts = {f'arr_{i}':acts[i].detach().numpy() for i in range(len(acts))}

## Reduce activations

In [ ]:
layer_to_reduced_acts = {layer:acts_util.reduce_activations(acts, dimensionality_reduction, dimension) for (layer,acts) in layer_to_acts.items()}


In [ ]:
layer_to_reduced_acts['arr_0'].shape

## Visualize activations

In [ ]:
layer_to_rgbs = {layer:vis_util.channels_to_rgbs(reduced_acts) for (layer,reduced_acts) in layer_to_reduced_acts.items()}


In [ ]:
layer_to_html = {}
for layer, rgbs in layer_to_rgbs.items():
    html = ''
    for tok, rgb in zip(tokens, rgbs):
        html += f"<span style='background-color: rgba({rgb[0]},{rgb[1]},{rgb[2]},1);'> {tok} </span>"
    layer_to_html[layer] = html

In [ ]:
for layer in layers:
    print(layer)
    _display_html(layer_to_html[layer])